## Warehouse-specific EOQ

In [9]:
def eoq(D, ordering_cost, holding_cost):
    return np.sqrt((2 * D * ordering_cost) / holding_cost)

## Warehouse-specific Reorder Point

In [4]:
from scipy.stats import norm
import numpy as np

def reorder_point(mean_d, std_d, lead_time, service_level=0.95):
    z = norm.ppf(service_level)
    return mean_d * lead_time + z * std_d * np.sqrt(lead_time)

## Multi-Warehouse Optimization Engine

In [6]:
import pandas as pd
# The eoq function is defined in this notebook, no need to import from src
# The reorder_point function is defined in this notebook, no need to import from src
# from src.simulation import simulate_inventory # Removed as it's not defined in this notebook or accessible

def optimize_all_warehouses(df):
    results = []

    for wh in df["warehouse"].unique():
        wdf = df[df["warehouse"] == wh]

        D = wdf["daily_demand"].sum()
        mean_d = wdf["daily_demand"].mean()
        std_d = wdf["daily_demand"].std()
        lead_time = wdf["lead_time"].iloc[0]

        Q = eoq(D, ordering_cost=300, holding_cost=2)
        ROP = reorder_point(mean_d, std_d, lead_time)

        # sim = simulate_inventory(wdf["daily_demand"], Q, ROP) # This line will cause an error if simulate_inventory is not defined

        results.append({
            "warehouse": wh,
            "EOQ": round(Q),
            "ROP": round(ROP)
            # "stockouts": sim["total_stockouts"],
            # "orders": sim["orders_placed"],
            # "ending_inventory": sim["ending_inventory"]
        })

    return pd.DataFrame(results)

## Main Execution (Network View)

In [11]:
import pandas as pd
# from src.multi_warehouse_optimizer import optimize_all_warehouses # Removed unnecessary import

df = pd.read_csv("data/multi_warehouse_demand.csv")

results = optimize_all_warehouses(df)
print(results)

print("\nNetwork-Level KPIs")
# print("Total Stockouts:", results["stockouts"].sum())
# print("Total Orders:", results["orders"].sum())
# print("Total Ending Inventory:", results["ending_inventory"].sum())

  warehouse   EOQ  ROP
0      WH_A  2543  262
1      WH_B  2095  267
2      WH_C  1473  200

Network-Level KPIs
